In [6]:
from model_training import ElevatorCallsDataset, ElevatorTCNModel
from torch.utils.data import DataLoader
import torch

from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
import numpy as np
from tqdm import tqdm

from livelossplot import PlotLosses


In [2]:
# load model
testset = pd.read_csv('testset.csv')
testset['timestamp'] = pd.to_datetime(testset['timestamp'])

# timestamp to unix timestamp
testset['timestamp'] = testset['timestamp'].astype(np.int64) // 10**9

test_dataset = ElevatorCallsDataset(testset, input_len=60, gap=30, output_window=60)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [3]:
model = ElevatorTCNModel(input_channels=len(testset.columns), output_size=len(testset.columns)-1)
model.to(device)
model.load_state_dict(torch.load('elevator_tcn_model.pth'))
model.eval()

all_preds = []
all_labels = []
accuracies = []
precisions = []
recalls = []

with torch.no_grad():
    model.eval()
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        logits = model(x_batch)
        preds = (torch.sigmoid(logits) > 0.5).float()
        
        all_preds.append(preds.cpu())
        all_labels.append(y_batch.cpu())


/tmp/ipykernel_2831/2849794424.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('elevator_tcn_model.pth'))


In [4]:
all_preds = torch.cat(all_preds, dim=0).numpy()
all_labels = torch.cat(all_labels, dim=0).numpy()


In [9]:
for i in tqdm(range(all_preds.shape[0])):
    accuracy = accuracy_score(all_labels[i], all_preds[i])
    precision = precision_score(all_labels[i], all_preds[i], zero_division=0)
    recall = recall_score(all_labels[i], all_preds[i], zero_division=0)
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    
print(f'Accuracy: {np.mean(accuracies)}')
print(f'Precision: {np.mean(precisions)}')
print(f'Recall: {np.mean(recalls)}')

100%|██████████| 513820/513820 [33:49<00:00, 253.20it/s]


Accuracy: 0.1753788619986739
Precision: 0.08375429110963774
Recall: 0.6562001903571381


In [12]:
df = pd.DataFrame(all_preds)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513815,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
513816,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
513817,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
513818,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
